In [1]:

pip install PyMySQL

Note: you may need to restart the kernel to use updated packages.


In [16]:
from bs4 import BeautifulSoup as bs
import urllib.request
import requests
import os
import pymysql
import uuid
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus
import re

In [3]:
juso_db = pymysql.connect(
    user='ssafy', 
    passwd='ssafy', 
    host='127.0.0.1', 
    db='eurekauser', 
    charset='utf8'
)

cursor = juso_db.cursor(pymysql.cursors.DictCursor)

In [4]:
def getanimalnum(pageNum):
    ##다운 받을 주소
    baseUrl = 'https://www.zooseyo.or.kr/Yu_board/freesale_ty_1.html?animal=%EA%B0%95%EC%95%84%EC%A7%80&area=&ty=1&page='   
    url = baseUrl+str(pageNum)  
    req=requests.get(url)
    print(req)
    html = req.text
    soup = bs(html, "html.parser")
    info=soup.select('a[href]')
    checkpoint=False
    ct=0;
    animalnum=[]
    for i in info:
        tags=i.get('href')
        if("sell_up.html?gort=g"==tags):
            ct+=1
        num=tags.split("&")[0].split("?")
        if(ct>1 and num[0]=="view.html"):
            animalnum.append(num[1].split("=")[1])
            
    return animalnum

In [5]:
def getinfo(animalNum):
    print("====개정보====")
    preUrl='https://www.zooseyo.or.kr/Yu_board/view.html?no='
    postUrl='&page=1&animal=&area='
    url=preUrl+animalNum+postUrl
    dogsinfo=[]
    
    req=requests.get(url)
    html = req.text
    soup = bs(html, "html.parser")
    
    #개번호 등록
    dogsinfo.append(animalNum)    
    
    #url 등록
    dogsinfo.append(url)

    
    ##img 경로등록
    imginfo=soup.select('img[src]')
    imglist=[]
    for i in imginfo:
        tags=i.get('src')
        if('/dog_sale/' in tags):
#             print(tags)
            imglist.append('https://www.zooseyo.or.kr'+tags)
    dogsinfo.append(imglist)
    
    ##개정보 등록
    infos=soup.text.split()
    
   ##날짜 인덱스 찾기
    for i in range(len(infos)):
        if"무료분양" in infos[i]:
            dogsinfo.append(infos[i].split("(")[1])
            break
    
    
    
    idx=0
    for idx in [356,364,368]:
        dogsinfo.append(infos[idx])
    

    
    
    return dogsinfo

In [28]:
def saveimg(directory,url,name="",dognum=""):
    if name.isalnum() or ":" in name :
        return 
    if "믹스" in name or "MIX" in name.capitalize():
        name = "믹스"
    filename=name+"_"+str(dognum)
    n=1
    imguuid=uuid.uuid1();
    save_path= directory+"/"+name
    save_root = directory+"/"+name
    if not os.path.exists(save_root):os.makedirs(save_root)
    try:
        with urlopen(url) as f:
            with open(os.path.join(save_path, filename+"_"+str(imguuid)+'.jpg'),'wb') as h: # w - write b - binary
                img = f.read()
                h.write(img)
        print(f"폴더명 :  { name} url : {url} 파일명 : {filename}")
        return [filename+"_"+str(imguuid)+'.jpg']
    except:
        print("error")
    

In [7]:
##데이터 추가 고민중
def saveAndInsert(dogInfos):
    return 

In [21]:
def check(s):
    
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') # 한글과 띄어쓰기를 제외한 모든 글자
    result = hangul.sub('', s) # 한글과 띄어쓰기를 제외한 모든 부분을 제거
    result=result.strip()
    if(result==""):
        return False
    else :
        return True



In [ ]:
if __name__ == "__main__":
    
    #크롤링 멈추는 날짜(등록일 기준)
    endDate="2020.03.15"  
    
    
    dogInfos=[]
    ##넘겨볼 페이지 갯수
    for pageNum in range(5):
        animalNums=getanimalnum(pageNum)
        for num in animalNums:
            info=getinfo(num)
            print(info)
            if(info[len(info)-4] in endDate):
                break
            dogInfos.append(info)
    print("####"*20)
    print("####"*20)
    print(f'Data Cnt: {len(dogInfos)}')
    print("####"*20)
    print("####"*20)
    
    
    
    
        
    

<Response [200]>
====개정보====
['337595', 'https://www.zooseyo.or.kr/Yu_board/view.html?no=337595&page=1&animal=&area=', ['https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1616030526_71012400.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1616030526_71012400.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1616030536_70353300.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1616030543_16895500.jpg'], '2021.03.18', '푸숑', '경기도', '+비숑']
====개정보====
['337590', 'https://www.zooseyo.or.kr/Yu_board/view.html?no=337590&page=1&animal=&area=', ['https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1616028719_63931200.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1616028719_63931200.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1616028719_81665200.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1616028719_84777500.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1616028719_87264500.jpg'], '2021.03.18', '동경이믹스견

['337497', 'https://www.zooseyo.or.kr/Yu_board/view.html?no=337497&page=1&animal=&area=', ['https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1615936051_04140800.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1615936051_04140800.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1615936051_20230900.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1615936051_23198000.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1615936051_26223800.JPG'], '2021.03.17', '믹스견', '서울', '/']
====개정보====
['337496', 'https://www.zooseyo.or.kr/Yu_board/view.html?no=337496&page=1&animal=&area=', ['https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1615935506_76099100.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1615935506_76099100.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1615935508_02656700.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1615935508_83645100.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103

['337510', 'https://www.zooseyo.or.kr/Yu_board/view.html?no=337510&page=1&animal=&area=', ['https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1615947568_39701100.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1615947568_39701100.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1615947568_52864200.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1615947568_54709300.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1615947568_56282700.jpg'], '2021.03.17', '저패니즈스피츠', '서울', '선택--']
====개정보====
['337499', 'https://www.zooseyo.or.kr/Yu_board/view.html?no=337499&page=1&animal=&area=', ['https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1615938871_18871600.png', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1615938871_18871600.png', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1615938871_41956900.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1615938871_43707000.png', 'https://www.zooseyo.or.kr/dog_sale/photo_free

['337443', 'https://www.zooseyo.or.kr/Yu_board/view.html?no=337443&page=1&animal=&area=', [], '2021.03.16', '말티즈', '광주', '/']
====개정보====
['337434', 'https://www.zooseyo.or.kr/Yu_board/view.html?no=337434&page=1&animal=&area=', ['https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1615875561_90884600.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1615875561_90884600.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1615875562_03353300.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1615875562_05782100.jpg'], '2021.03.16', '말티즈', '서울', '선택--']
====개정보====
['337428', 'https://www.zooseyo.or.kr/Yu_board/view.html?no=337428&page=1&animal=&area=', ['https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1615871284_03334900.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1615871284_03334900.jpg', 'https://www.zooseyo.or.kr/dog_sale/photo_free/202103/1615871284_21292100.jpg'], '2021.03.16', '푸들', '전국분양', 'mh8908@hanmail.net']
====개정보====
['337425'

In [26]:
for info in dogInfos:
    if(info[2] == []) :
        continue
    for img in info[2][1:]:
        ##요기에 원하는 경로
        saveimg("C:/Users/sskim/Downloads/Dogs",img,info[len(info)-3],info[0])
    
            

OSError: [WinError 123] 파일 이름, 디렉터리 이름 또는 볼륨 레이블 구문이 잘못되었습니다: 'C:/Users/sskim/Downloads/Dogs/:'